# IMPORT


In [9]:
import os, json, pickle, re, math, torch, multiprocessing, nltk
from tqdm import tqdm
import numpy as np

import wikipedia
from joblib import Parallel, delayed 
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import networkx as nx
from pyvis.network import Network
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# OFFLINE WIKI CLASS

In [10]:
# IMPORTING
from wikiDump_cleaner import Cleaner
import bz2, os, re, sys, json, pickle
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from tqdm import tqdm
from difflib import SequenceMatcher
# from joblib import Parallel, delayed 


class offline_Wiki():
    def __init__(self, 
                 wiki_index_file = None,
                 wikiDump_bz2_file = None,
                 index_folder = None,
                 verbose = False
                 ):
        self.verbose = verbose
        self.wiki_index_file = wiki_index_file
        self.wikiDump_bz2_file = wikiDump_bz2_file
        
        self.prefixx = "index_"
        self.suffixx = ".p" 
        self.files_index = []
        self.index_keys = []
        self.index_folder = index_folder

        if self.index_folder:
            self.files_index = os.listdir(index_folder)
            self.index_keys = sorted([".".join(os.path.basename(i).split(".")[:-1]).split("_")[1] for i in self.files_index], key=lambda x: x.lower())

        else:
            _choice = input("Wiki index folder for is not provided. Do you want to create index ? (1/0) : ").strip()
            if _choice in "10" and len(_choice) == 1 :
                if int(_choice):
                    self.index_folder = self.index_maker(wiki_index_file, wikiDump_bz2_file, verbose=self.verbose)
                    self.files_index = os.listdir(self.index_folder)
                    self.index_keys = sorted([".".join(os.path.basename(i).split(".")[:-1]).split("_")[1] for i in self.files_index], key=lambda x: x.lower())
                else:
                    print("Index folder not made ...")
    ###
    ### --- INDEX MAKER ---
    ###
                    
    def get_start_bytes_list(self, txt_wiki_dump_index_path, bz2_wiki_dump_path, verbose = False):
        start_bytes = []
 
        with open(txt_wiki_dump_index_path, "r", encoding="utf-8") as f:
            index_file = f.readlines()
        
        if verbose:
            start_bytes = [int(x.split(":")[0]) for x in tqdm(index_file)] # <--- SIMPLY USING A for LOOP IS VERY VERY FAST (7 SEC)
            # start_bytes = Parallel(n_jobs=N_JOB_COUNT)(delayed(get_start_bytes_list_helper)(x) for x in tqdm(index_file)) # <--- USING Parallel FINISHES IN >3 MINS 😮
        else:
            start_bytes = [int(x.split(":")[0]) for x in index_file]

        # to deduplicate the list
        start_bytes = list(set(start_bytes))  #
        
        # but we want them in a specific order

        file_size = os.path.getsize(bz2_wiki_dump_path)
        start_bytes.append(file_size + 1)

        start_bytes.sort()

        if verbose:
            print(f"GOT {len(start_bytes)} START BYTES")
        
        return start_bytes
    
    def acceptableWord(self, word, verbose = False):
        flag = False
    
        if verbose:
            print(f"Checking {word}", end="\r")
    
        if any((i in word) for i in ["File:", 
                                    "Template:", 
                                    "Wikipedia:", 
                                    "Category:", 
                                    "Help:", 
                                    "Portal:",
                                    "MediaWiki:",
                                    "Draft:",
                                    "Module:"]):
            return flag
        
        if any([word.endswith(ext) for ext in [".jpg",
                                                ".png", 
                                                ".gif", 
                                                ".zip", 
                                                ".ogg", 
                                                ".mp3", 
                                                ".mp4", 
                                                ".webp"]]):
            return flag
        
        flag = True
    
        return flag

    def clean_filename(self, filename):
        # Remove invalid characters for filenames
        return re.sub(r'[^\w\-_.() ]', ' ', filename)
    
    def save_pickle(self, dataa, filename):
        with open(filename, "wb") as f:
            pickle.dump(dataa, f)

    def store_dictionary_in_bins(self, word_dictionary, binsize=10000, 
                                index_folder ="./indexes/", verbose = False):
        if not os.path.exists(index_folder):
            os.makedirs(index_folder)

        sorted_keys = sorted(word_dictionary.keys())
        # sorted_keys = sorted(word_dictionary.keys(), key=lambda x: x.lower())  # Sort keys case-insensitively
        num_bins = len(sorted_keys) // binsize + (1 if len(sorted_keys) % binsize != 0 else 0)

        for i in tqdm(range(num_bins)):
            start_idx = i * binsize
            end_idx = min((i + 1) * binsize, len(sorted_keys))
            bin_keys = sorted_keys[start_idx:end_idx]
            bin_data = {key: (word_dictionary[key]) for key in bin_keys}

            first_key = bin_keys[0]
            filename = f"index_{self.clean_filename(first_key).strip()}.p"
            # Check if the cleaned filename is less than 3 characters
            if len(filename) < 8+2 and i > 0:
                # Try using the second key in bin_keys as the filename
                # if len(bin_keys) > 1:
                #     second_key = bin_keys[1]
                #     cleaned_filename = clean_filename(f"index_{second_key}.json")
            # if len(filename) < 8+3:
                print(i, filename, first_key , len(filename))
                # print(cleaned_filename, second_key , len(cleaned_filename))
                continue

            file_path = index_folder+filename
            self.save_pickle(bin_data, file_path)
            # # with open(file_path, 'w') as f:
            # #     json.dump(bin_data, f, indent=3)
            if verbose:
                print(f"Stored {len(bin_data)} elements in {filename}")

    def index_maker(self, index_file, wikiDump, index_folder = "./indexes/", verbose = False):
        if not index_file:
            index_file = self.wiki_index_file
        
        if not wikiDump :
            wikiDump = self.wikiDump_bz2_file
        
        # wiki_bz2_file_size = os.path.getsize(wikiDump)
        
        start_byte_list = self.get_start_bytes_list(index_file, wikiDump, verbose=verbose)
        
        if verbose:
            print("making start - end bytes list")
        start_end_list = [(start_byte_list[i], start_byte_list[i+1]) for i in range(len(start_byte_list)-1)]

        start_byte_list.clear() # EMPTYING MEMEORY
        
        if verbose:
            print("start - end bytes list made")

        start_end_dict = dict(start_end_list)

        word_start_end_dict = {}       
        if verbose : 
            print(f"Opening {index_file}.")

        with open(index_file, "r", encoding = "utf-8") as f:
            index_file = f.readlines()

        if verbose:
            print(f"length of index file : {len(index_file)}")

        for line in tqdm(index_file):
            start_byte, _idk, word = line.split(":")[0], line.split(":")[1], ":".join(line.split(":")[2:]).strip() 
            start_byte = int(start_byte)
            # word = ":".join(word)
            # if word 
            # print(f"{start_byte}, {start_end_dict[start_byte]} : {word}")
            if not self.acceptableWord(word):
                # print(word)
                continue
            word_start_end_dict[word] = (start_byte, start_end_dict[start_byte])
            # input()
            # start_byte = 
        # start_bytes = [int(x.split(":")[0]) for x in tqdm(index_file)] # <--- SIMPLY USING A for LOOP IS VERY VERY FAST (7 SEC)
        if verbose:
            print(f"Length of word - (start, end) dict is : {len(word_start_end_dict)}")

        index_file.clear() # EMPTYING MEMEORY

        if verbose:
            print("Making bins")
        self.store_dictionary_in_bins(word_start_end_dict, binsize=20000, index_folder=index_folder) # STORING IN BINS
        
        if verbose:
            print("Bins made")
        
        return index_folder

    ###
    ### --- INDEX READER ---
    ###

    def load_pickle(self, filename):
        res = None
        with open(filename, "rb") as f:
            res = pickle.load(f)
        return res
    
    ### ----- SIMILARITY FINDING FUNCTIONS | STARTS -----

    def search_closest_words(self, keywords, word):
        word_lower = word.lower()  # Convert search word to lowercase
        start = 0
        end = len(keywords) - 1
        closest_words = []

        while start <= end:
            mid = (start + end) // 2

            # Convert current keyword to lowercase for comparison
            keyword_mid_lower = keywords[mid].lower()

            # Check if the word falls between keywords[mid] and keywords[mid+1]
            if keyword_mid_lower < word_lower < keywords[mid + 1].lower():
                closest_words.append(keywords[mid])
                closest_words.append(keywords[mid + 1])
                break
            elif word_lower < keyword_mid_lower:
                end = mid - 1
            else:
                start = mid + 1

        return closest_words

    def jaccard_similarity_word(self, s1, s2):
        set1 = set(s1.lower())  # Convert s1 to lowercase
        set2 = set(s2.lower())  # Convert s2 to lowercase
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union != 0 else 0

    def jaccard_similarity(self, str1, str2):
    # Convert input strings to sets of words
        set1 = set(str1.lower().split())
        set2 = set(str2.lower().split())
        
        # Calculate Jaccard similarity
        intersection = len(set1 & set2)
        union = len(set1 | set2)
        # similarity = intersection / union
        
        # return similarity
        return intersection / union if union != 0 else 0

    def weighted_jaccard_similarity(self, s1, s2):
        set1 = set(s1.lower().split())  # Convert to lowercase and split into words
        set2 = set(s2.lower().split())
        # intersection_weight = sum(min(list(set1).count(w), list(set2).count(w)) for w in set1.intersection(set2))
        # union_weight = sum(max(list(set1).count(w), list(set2).count(w)) for w in set1.union(set2))
        intersection_weight = len(set1.intersection(set2))
        union_weight = len(set1.union(set2))
        return intersection_weight / union_weight if union_weight != 0 else 0
        
    def is_fuzz_similar(self, string1, string2, threshold = 80, verbose = False):
        ratioo = fuzz.ratio(string1, string2)
        if verbose:
            print(f"Fuzzy similarity between {string1} and {string2} is {ratioo}")
        if ratioo >= threshold:
            return True 
        return False 

    ### ----- SIMILARITY FINDING FUNCTIONS | ENDS -----

    def find_similar_keys(self, word, dictionary, threshold=0.5, verbose=False):
        similar_keys = []

        if verbose:
            print(f"Finding {word}...")

        word_lower = word.lower()  # Convert word to lowercase

        for key in dictionary:
            # similarity = fuzz.ratio(word_lower, key)
            # similarity = self.jaccard_similarity_word(word_lower, key)  # Use lowercase word for comparison
            similarity = self.jaccard_similarity(word_lower, key)  # Use lowercase word for comparison
            # similarity = weighted_jaccard_similarity(word_lower, key)  # Using weighted Jaccard
            if similarity > threshold:
                similar_keys.append((key, similarity))

        similar_keys.sort(key=lambda x: x[1], reverse=True)  # Sort by similarity
        similar_keys = [key[0] for key in similar_keys]  # Extract keys only

        return similar_keys if similar_keys else None

    def fetch_word_from_list(self, target_word, similar_words, threshold = 0.7): 
        for word in similar_words:
            if word == target_word:
                return word
        for word in similar_words:    
            if word.lower() == target_word.lower():
                return word
        
        similarr = []
        word_lower = word.lower()
        for key in similar_words:
            # _similarity = fuzz.ratio(word_lower, key)
            _similarity = self.jaccard_similarity(word_lower, key)  # Use lowercase word for comparison

            # _similarity = self.weighted_jaccard_similarity(word_lower, key)  # Use lowercase word for comparison
            # similarity = weighted_jaccard_similarity(word_lower, key)  # Using weighted Jaccard
            if _similarity > threshold:
                similarr.append((key, _similarity))

        similarr.sort(key=lambda x: x[1], reverse=True)  # Sort by similarity
        similarr = [key[0] for key in similarr]  # Extract keys only
        
        return similarr[0] if similarr else None
        # return find_most_similar_word(target_word, similar_words, max_similarity=threshold)  
        # return None

    def find_most_similar_word(self, query, keywords, max_similarity = 0.7):
        # max_similarity = 0
        most_similar_word = None

        # Iterate through the keywords and find the most similar one to the query
        for keyword in keywords:
            similarity = SequenceMatcher(None, query, keyword).ratio()
            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_word = keyword

        return most_similar_word


    def page_cleaner(self, page_body, summaryOnly = False):
        cleaner = Cleaner()
        cleaned_page = cleaner.clean_text(page_body)
        cleaned_text, links = cleaner.build_links(cleaned_page)

        if summaryOnly:
            cleaned_text = cleaned_text.split("==")[0] # THE FIRST PARAGRAPH/SECTION HAS THE SUMMARY

        return cleaned_text, links

    def extract_cleaned_page(self, page_soup, summaryOnly = False, verbose = False, wantLinks = False, wikibaseurl = "https://en.wikipedia.org/wiki/"):
        """
        RETURNS THE PAGE TITLE, URL, AND PAGE CONTENT
        YOU MAY WANT TO CHANGE THIS ACCORDING IT YOUR NEED
        
        """
        # print(page_soup.find("title"))
        # input()
        page_title = page_soup.find("title").text
        page_body = page_soup.find("text").text
        page_redirect = page_soup.find("redirect") 
        page_url = wikibaseurl+page_title.replace(' ', '_')

        if page_redirect:
            page_redirect = page_redirect['title']
            page_url = wikibaseurl+page_redirect.replace(' ', '_')
            # page_title = page_redirect.replace(' ', '_')
            
        page_body, _ = self.page_cleaner(page_body, 
                                    summaryOnly=summaryOnly)
    
        if verbose:
            print(f"Page cleaning done... \nGot Title : {page_title}, \nCleaned page body : {page_body} \nPage url : {page_url}, {'and Links : {_}'*wantLinks}\n")
        
        returning = [page_title, page_url, page_body]

        if wantLinks:
            returning.append(_)

        return returning

    def extract_pages(self, page_xml):
        soup = BeautifulSoup(page_xml, "lxml")
        pages = soup.find_all("page")
        return pages
    
    # def retrieve_text(self, title, offset):
    def decompress_xml2(self, title, offset):

        '''
        TAKEN FROM : "https://gerrit.wikimedia.org/r/plugins/gitiles/operations/dumps/+/ariel/toys/bz2multistream/wikiarticles.py"

        retrieve the page text for a given title from the xml file
        this does decompression of a bz2 stream so it's more expsive than
        other parts of this class
        arguments:
        title  -- the page title, with spaces and not underscores, case sensitive
        offset -- the offset in bytes to the bz2 stream in the xml file which contains
                  the page text
        returns the page text or None if no such page was found
        '''
        # self.xml_fd.seek(offset)
        with open(self.wikiDump_bz2_file, "rb") as f:
            f.seek(offset)

            unzipper = bz2.BZ2Decompressor()
            out = None
            found = False
            try:
                # block = self.xml_fd.read(262144)
                block = f.read(262144)

                out = unzipper.decompress(block).decode()
            # hope we got enough back to have the page text
            except:
                raise
            # format of the contents (and there are multiple pages per stream):
            #   <page>
            #   <title>AccessibleComputing</title>
            #   <ns>0</ns>
            #   <id>10</id>
            # ...
            #   </page>
            title_regex = re.compile(r"<page>(\s*)<title>%s(\s*)</title>" % re.escape(title))
            while not found:
                match = title_regex.search(out)
                if match:
                    found = True
                    text = out[match.start():]
                    if self.verbose:
                        sys.stderr.write("Found page title, first 600 characters: %s\n" % text[:600])
                    break
                # we could have a part of the regex at the end of the string, so...
                if len(out) > 40 + len(title):  # length of the above plus extra whitespace
                    out = out[-1 * (40 + len(title)):]
                try:
                    # block = self.xml_fd.read(262144)
                    block = f.read(262144)

                except:
                    # reached end of file (normal case) or
                    # something really broken (other cases)
                    break
                try:
                    out = out + unzipper.decompress(block).decode()
                except EOFError:
                    # reached end of bz2 stream
                    # EOFError  means we have some data after end of stream, don't care
                    pass
            if not found:
                return None
            out = text
            found = False
            text = ""
            while not found:
                ind = out.find("</page>")
                if ind != -1:
                    found = True
                    if self.verbose:
                        sys.stderr.write("Found end page tag\n")
                    text = text + out[:ind + len("</page>")]
                    break
                # we could have part of the end page tag at the end of the string
                text = text + out[:-1 * len("</page>") - 1]
                out = out[-1 * len("</page>"):]
                try:
                    # block = self.xml_fd.read(262144)
                    block = f.read(262144)

                except:
                    # reached end of file (normal case) or
                    # something really broken (other cases)
                    break
                try:
                    out = out + unzipper.decompress(block).decode()
                except EOFError:
                    # reached end of bz2 stream
                    # EOFError  means we have some data after end of stream, don't care
                    pass
            # if not found this can be partial text. should we return it? no
            if not found:
                if self.verbose:
                    sys.stderr.write("Found partial text but no end page tag. Text follows:\n")
                    sys.stderr.write(text)
                    sys.stderr.write("\n")
                text = None
            return text

    def decompress_xml(self, bz2_wiki_dump_path, start_byte, end_byte, verbose = False):
        decomp = bz2.BZ2Decompressor()
        with open(bz2_wiki_dump_path, 'rb') as f:
            f.seek(start_byte)
            block_size = end_byte - start_byte - 1
            print(block_size)
            input()
            # readback = f.read(end_byte - start_byte - 1)
            readback = f.read(max(256*1024, block_size) + 256*1024)

            page_xml = decomp.decompress(readback).decode()

            pages = self.extract_pages(page_xml)
            
            if verbose:
                print(f"FOUND : {len(pages)} PAGES BETWEEN {start_byte} BYTE AND {end_byte} BYTE.")
            
        return pages 
    
    def word_match(self, word, verbose = False, summaryOnly = True):
        near_words = self.search_closest_words(self.index_keys, word)
        if verbose:
            print(f"near words similar to {word} : {near_words}")
        near_words_file_path = [self.index_folder + self.prefixx + word + self.suffixx for word in near_words]
        
        to_search_into = {}
        [to_search_into.update(self.load_pickle(file_pathh)) for file_pathh in near_words_file_path]
        similar_keywords = self.find_similar_keys(word, to_search_into)
        # similar_keywords = self.find_most_similar_keys(word, to_search_into)

        if verbose:
            print(f"Similar keywords found in Wiki : {similar_keywords}")
        
        if not similar_keywords:
            if verbose:
                print("No similar keyword found !!!")
            return None
  
        wanted = self.fetch_word_from_list(word, similar_keywords)
        
        if not wanted:
            wanted = similar_keywords[0]
        
        if verbose:
            print(f"Wanted : {wanted}")

        _start, _end = to_search_into[wanted]
        
        if verbose:
            print(f"Byte start : {_start}, Byte end : {_end}")
        
        # decompressed_pages = self.decompress_xml(self.wikiDump_bz2_file, _start, _end)
        page_xml = self.decompress_xml2(wanted, _start)
        decompressed_pages = BeautifulSoup(page_xml, "lxml").find_all("page")

        # print(decompressed_pages)
        # input()


        for page_xml in decompressed_pages:
            # print(page_xml)
            _page_title,_page_url, _page_summary = "", "", ""
            _page_title, _page_url, _page_summary = self.extract_cleaned_page(page_xml, summaryOnly=summaryOnly, verbose=verbose)
            if self.is_fuzz_similar(wanted, _page_title, threshold=90,verbose=verbose): 
                # offline_dict[_page_title] = {'title' : _page_title, 
                #                             'url' : _page_url, 
                #                             'summary' : _page_summary}
                return {'title' : _page_title, 
                        'url' : _page_url, 
                        'summary' : _page_summary}
        return None
        

# KB CLASS 

In [11]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add
            

    def get_wikipedia_data(self, candidate_entity, useWiki = True, offline_wiki = None, verbose = False):
        # print("\n\n--- offline", offline_Wiki)
        entity_data = None
        stop_words = set(stopwords.words('english'))
        if len(candidate_entity.split()) > 4:
            word_tokens = word_tokenize(candidate_entity)
            candidate_entity = " ".join([w for w in word_tokens if not w.lower() in stop_words])

        try:
            if offline_wiki:
                if verbose:
                    print(f"Finding {candidate_entity} in offline Wiki")
                _entity_data = offline_wiki.word_match(candidate_entity, verbose = verbose)
                
                if verbose:
                    print(f"Got {_entity_data} after word_match from offline Wiki")

                if "REDIRECT".lower() in _entity_data["summary"][:10].lower():
                    # entity_data = _entity_data
                    _word = _entity_data["url"].split("/wiki/")[-1].strip()
                    if verbose:
                        print(f"REDIRECT found !!! Candidate entitiy {candidate_entity} === changed to ==> {_word}")

                    entity_data = self.get_wikipedia_data(_word, useWiki=useWiki, offline_wiki=offline_Wiki, verbose=verbose)
                else:                    
                    ratioo = fuzz.ratio(candidate_entity, _entity_data['title'])
                    if verbose:
                        print(f"Fuzz ration : {ratioo}")
                    if ratioo > 50 :
                        entity_data = _entity_data
                        if verbose:
                            print(f"Got {entity_data} from offline wiki with similarity ration = {ratioo}.")
            
            if useWiki and not entity_data:
                if verbose:
                    print(f"Finding {candidate_entity} in online Wiki")
                page = wikipedia.page(candidate_entity, auto_suggest=False)
                entity_data = {
                    "title": page.title,
                    "url": page.url,
                    "summary": page.summary
                }
            
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r, article_title, article_publish_date, 
                     useWiki = True, offlineWiki = None, verbose = False):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        if verbose:
            print(f"Candidate entities : {candidate_entities}")
            
        # entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]
        
        # TRY 2
        entities = []
        if useWiki:
            entities = Parallel(n_jobs=N_JOB_COUNT)(delayed(self.get_wikipedia_data)(ent, useWiki, offlineWiki, verbose=verbose) for ent in candidate_entities)
            # entities = [self.get_wikipedia_data(ent, useWiki, offlineWiki) for ent in candidate_entities]

        else:
            entities = [{"title": ent,
                         "url": "",
                         "summary": ""
                        } for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")

# FOR GPU

In [12]:
def get_cpu_count():
    c = multiprocessing.cpu_count()
    return c 

def check_gpu():
    for i in range(torch.cuda.device_count()):
        device_name = f'cuda:{i}'
        print(f'{i} device name:{torch.cuda.get_device_name(torch.device(device_name))}')

def get_gpu():
    return [f'cuda:{i}' for i in range(torch.cuda.device_count())]

print(check_gpu())

print(get_gpu())

print(get_cpu_count())

N_JOB_COUNT = get_cpu_count()//2
N_JOB_COUNT = 1

0 device name:NVIDIA GeForce RTX 3060 Laptop GPU
None
['cuda:0']
16


# HELPER FUNCTIONS

In [13]:
def loadJSON(filepathh):
    _dataa = {} 
    if os.path.exists(filepathh):
        with open(filepathh, "r", encoding="utf-8") as _f:
            _dataa = json.load(_f)
    else:
        print(f"{filepathh} does not exists...\n") 
    return _dataa 

def loadTXT(filepathh):
    _dataa = ""
    if os.path.exists(filepathh):
        with open(filepathh, "r", encoding="utf-8") as _f:
            _dataa = _f.read()
    else:
        print(f"{filepathh} does not exists...\n") 
    return _dataa 

def loadFILE(filepathh = ""):
    if os.path.exists(filepathh):
        if filepathh.endswith(".txt"):
            return loadTXT(filepathh)
        elif filepathh.endswith(".json"):
            return loadJSON(filepathh)
        else:
            print("\n- Invalid File format 😐 !!!\n")
            return None
    else:
        print(f"{filepathh} does not exists...\n") 

def remove_garbage(text):
    # Remove garbage Unicode characters
    cleaned_text = text.encode().decode('unicode-escape')
    # Remove any remaining non-printable characters
    cleaned_text = re.sub(r'[^\x20-\x7E]', '', cleaned_text)
    return cleaned_text

def clean_sentence(sentence):
    # Remove extra white spaces
    cleaned_sentence = re.sub(r'\s+', ' ', sentence)
    # Remove unwanted characters except alphabets, numbers, punctuation marks, '@', '-', and '_'
    cleaned_sentence = re.sub(r'[^a-zA-Z0-9@#\-_.,?!\'" ]', '', cleaned_sentence)
    # Remove words containing '#' and 'pic.twitter.com'
    cleaned_sentence = ' '.join(word if '#' not in word and 'pic.twitter.com' not in word else ' ' for word in cleaned_sentence.split() )
    return cleaned_sentence.strip()

def clean_document(document):
    document = remove_garbage(document)
    # Tokenize the document into sentences
    sentences = sent_tokenize(document)
    # Clean each sentence
    cleaned_sentences = [clean_sentence(sentence) for sentence in sentences]
    return cleaned_sentences

In [14]:
def save_network_html(kb, filename="network.html", 
                      verbose = False, 
                      physics = False,
                      show = False):

    if not os.path.exists(filename):
        with open(filename, 'w') as _file:
            _file.write("")

    # create network
    G = nx.Graph()
    net = Network(directed=True, 
                  notebook=True,
                  width="1000px", 
                  height="1000px",
                #   bgcolor="#eeeeee"
                )
    if verbose:
        print("Network initialized")

    # nodes
    color_entity = "#00FF00"
    if verbose:
        print(f"there are {len(kb.entities)} entities in KB")
    for e in kb.entities:
        G.add_node(e)
        net.add_node(e, label=e, shape="dot", color=color_entity)
        # net.add_node(e, label=e, physics = physics, shape="dot", color=color_entity)
    
    # edges
    if verbose:
        print(f"there are {len(kb.relations)} relations in KB")
    
    # for r in kb.relations:
    #     G.add_edge(r['head'], r["tail"], )
    #     # net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])
    labels = {}
    for r in kb.relations:
        G.add_edge(r['head'], r["tail"])
        labels[(r["head"], r["tail"])] = r["type"]
        net.add_edge(r["head"], r["tail"], title=r["type"], label=r["type"])

    scale=10 # Scaling the size of the nodes by 10*degree
    d = dict(G.degree)

    pos = nx.spring_layout(G)
    #Updating dict
    d.update((x, scale*y) for x, y in d.items())

    #Setting up size attribute
    nx.set_node_attributes(G,d,'size')
    nx.set_edge_attributes(G,labels, 'labels')
    # nx.draw_networkx_edge_labels(
    #                             G, pos,
    #                             edge_labels=labels,
    #                             # font_color='red'
    #                             )
    if verbose:
        print(f"Trying to make graph")

    # net.from_nx(G)   
    
    # save network
    if physics:
        net.repulsion(
            node_distance=200,
            central_gravity=0.3,
            spring_length=200,
            spring_strength=0.05,
            damping=0.09
        )

    net.set_edge_smooth('dynamic')

    if verbose:
        print(f"Trying to show graph")

    net.show(filename)

def save_kb(kb, filename, verbose = False):
    if verbose:
        print(f"there are {len(kb.entities)} entities in KB")
        print(f"there are {len(kb.relations)} relations in KB")

    with open(filename, "wb") as f:
        pickle.dump(kb, f)

def load_kb(filename):
    res = None
    with open(filename, "rb") as f:
        res = pickle.load(f)
    return res


# DOING NER

In [15]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [16]:
def split_docs(text, 
               max_text_count = 1000,
               verbose = False):
    
    sentences = sent_tokenize(text)
    chunks = []
    chunk = ""
    len_sentence = 0
    chunk_len = 0
    for sentence in sentences:
      len_sentence = len(sentence.strip().split())
      if chunk_len+len_sentence < max_text_count:
         chunk += sentence+" "
         chunk_len += len_sentence
         continue
      chunk_len = len_sentence
      chunks.append(chunk)
      chunk = sentence
      
      # chunk = textwrap.wrap(sentence, max_text_count)
    if verbose:
      print(len(chunks))
    return chunks

def _from_text_to_kb(text, article_url, kb = None,
                    useGPU=0, 
                    span_length=128, 
                    article_title=None,
                    article_publish_date=None, 
                    verbose=False,
                    useWiki=True,
                    offline_Wiki = None):
    
    # tokenize whole text
    # print(text)
    # input()
    with torch.no_grad():
        inputs = tokenizer([text], 
                        max_length = 1000,
                        #    max_length=512,
                        padding=True,  
                        truncation=True, 
                        return_tensors="pt")

        # compute span boundaries
        # print(inputs.values())
        num_tokens = len(inputs["input_ids"][0])
        if verbose:
            print(f"Input has {num_tokens} tokens")
        num_spans = math.ceil(num_tokens / span_length)
        
        if verbose:
            print(f"Input has {num_spans} spans")
        overlap = math.ceil((num_spans * span_length - num_tokens) / 
                            max(num_spans - 1, 1))
        
        # input()
        spans_boundaries = []
        start = 0
        for i in range(num_spans):
            spans_boundaries.append([start + span_length * i,
                                    start + span_length * (i + 1)])
            start -= overlap
        if verbose:
            print(f"Span boundaries are {spans_boundaries}")

        # transform input with spans
        tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
        tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                        for boundary in spans_boundaries]
        inputs = {
            "input_ids": torch.stack(tensor_ids),
            "attention_mask": torch.stack(tensor_masks)
        }
        
        # generate relations
        num_return_sequences = 3
        gen_kwargs = {
            "max_length": 256,
            "length_penalty": 0,
            "num_beams": 3,
            "num_return_sequences": num_return_sequences
        }

        generated_tokens = model.generate(
                                            inputs["input_ids"].to(model.device),
                                            attention_mask=inputs["attention_mask"].to(model.device),
                                            **gen_kwargs,
                                            )

        # decode relations
        decoded_preds = tokenizer.batch_decode(generated_tokens,
                                            skip_special_tokens=False)

        # create kb
        if not kb:
            kb = KB()

        i = 0
        # for sentence_pred in tqdm(decoded_preds, leave=False):
        _relations = Parallel(n_jobs=N_JOB_COUNT)(delayed(extract_relations_from_model_output)(sentence_pred) for sentence_pred in decoded_preds)

        for sentence_pred in decoded_preds:
            current_span_index = i // num_return_sequences
            # relations = extract_relations_from_model_output(sentence_pred)
            relations = _relations[i]

            if verbose:
                print(f"{i}. extraction of relations done, it has {len(relations)} relations", end="\r")
                
            for relation in relations:
                relation["meta"] = {
                    article_url: {
                        "spans": [spans_boundaries[current_span_index]]
                    }
                }
                kb.add_relation(relation, 
                                article_title,
                                article_publish_date, 
                                useWiki=useWiki,
                                offlineWiki=offline_Wiki,
                                verbose=verbose)
            i += 1

    return kb

def from_text_to_kb(text, article_url, kb = None,
                    useGPU=0, 
                    span_length=128, 
                    article_title=None,
                    article_publish_date=None, 
                    verbose=False,
                    max_token = 1000,
                    max_doc_text = 1000,
                    useWiki = True,
                    offlineWiki = None):
    # with torch.no_grad():
    #     # tokenize whole text
    #     # inputs = tokenizer([text], return_tensors="pt")
    #     # num_tokens = len(inputs["input_ids"][0])

    input_words = text.split()
    num_tokens = len(input_words)

    if verbose:
        # print(f"Input has {num_tokens} tokens")
        print(f"Input has {num_tokens} words")

    if not kb:
        kb = KB()
    
    _kb = kb 

    _offlineWiki = offlineWiki

    # compute span boundaries
    # num_tokens = len(inputs["input_ids"][0])
    if num_tokens > max_token:
        if verbose:
            print("input len > token size, splitting doc in smaller chunks")
        text = split_docs(text, max_text_count=max_doc_text)
    
    if type(text) == str:
        text = [text]
    
    # for _text in tqdm(text, leave=False):
    for _text in text: 
        # print(_text)
        # print(_text[0])
        # input()
        _kb = _from_text_to_kb(_text, article_url, 
                            useGPU=useGPU, 
                            span_length=span_length, 
                            article_title=article_title,
                            article_publish_date=article_publish_date, 
                            verbose=verbose,
                            kb=_kb,
                            useWiki=useWiki,
                            offline_Wiki=_offlineWiki)
    return _kb
                

# -- BREAK --

# EXP

In [17]:
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")


### LOADING MODEL

In [18]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

In [19]:
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50272, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50272, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

### LOADING OFFLINE WIKI

In [20]:
WIKI_INDEX_FILE = "D://WikiDump/enwiki-20240220-pages-articles-multistream-index.txt/enwiki-20240220-pages-articles-multistream-index.txt"
WIKI_BZ2_FILE = "D://WikiDump/enwiki-20240220-pages-articles-multistream.xml.bz2"

INDEX_FOLDER = "./indexes/"

In [21]:
offline_wikipedia = offline_Wiki(wiki_index_file=WIKI_INDEX_FILE,
                                wikiDump_bz2_file=WIKI_BZ2_FILE, 
                                index_folder=INDEX_FOLDER,
                                verbose=False,)

In [22]:
# testing
print(offline_wikipedia.word_match("tigers", summaryOnly=False))
# input()

test_words = ["sachin", "trump", "wedding", "white house", "modi", "apple", "windows 11"]
for test_word in test_words:
    print(offline_wikipedia.word_match(test_word, summaryOnly=False))

{'title': 'Tigers', 'url': 'https://en.wikipedia.org/wiki/Tiger', 'summary': 'REDIRECT Tiger'}
{'title': 'Sachin', 'url': 'https://en.wikipedia.org/wiki/Sachin', 'summary': 'Sachin may refer to:\nSachin (given name), an Indian given name, including a list of people with the name\n* Sachin (actor) (born 1957), Indian actor and filmmaker\n* Sachin (boxer), Indian boxer\n* Sachin Tendulkar (born 1973), Indian cricketer\n==Films==\nSachein, a 2005 Tamil film directed by John Mahendran, sometimes spelled "Sachin"\n A Billion Dreams, a 2017 Indian biographical film of Sachin Tendulkar\nSachin (film), a 2018 Malayalam film directed by Santhosh Nair\n The Ultimate Winner, a 2023 Indian film based on the cricketer\n==Places==\nSachin, Pas-de-Calais, a town in northern France\nSachin, Gujarat, a suburban area of Surat in India\n*Sachin INA, a town and an industrial notified area\n* Sachin railway station, a small railway station in Surat district, Gujarat\nSachin State, a princely state of India

### TESTING


In [15]:
statement = "The Sun rises in East direction. The Earth is smaller than the planet Jupiter. Tigers are from Cat family."

In [23]:
statement = "We actually saw revenues to the Treasury increase after we lowered taxes in 2017. Rest assured the Democrats do not want you to know that."

In [24]:
# texts = statement.split('. ')
texts = sent_tokenize(statement)
texts

['We actually saw revenues to the Treasury increase after we lowered taxes in 2017.',
 'Rest assured the Democrats do not want you to know that.']

In [25]:
testing_texts = texts

In [26]:
# kb = KB()
# max_lenn = 1000
# spann = 64

# kb = from_text_to_kb(statement, "", kb = kb,
#                     verbose=0,
#                     span_length=spann,
#                     max_doc_text=max_lenn,
#                     useWiki=1,
#                     offlineWiki=offline_wikipedia)
# kb.print()


In [30]:
kb = KB()
max_lenn = 1000
spann = 64

for text in tqdm(texts):
    print(f"checking - '{text}'")
    kb = from_text_to_kb(text, "", kb = kb,
                         verbose=1,
                         span_length=spann,
                         max_doc_text=max_lenn,
                         useWiki=1,
                         offlineWiki=offline_wikipedia)
kb.print()
    

  0%|          | 0/2 [00:00<?, ?it/s]

checking - 'We actually saw revenues to the Treasury increase after we lowered taxes in 2017.'
Input has 14 words
Input has 17 tokens
Input has 1 spans
Span boundaries are [[0, 64]]
Candidate entities : ['lowered taxes in 2017', '2017']
Finding lowered taxes in 2017 in offline Wiki
near words similar to lowered taxes in 2017 : ['Louvilliers-les-Perche', 'Loyalton  Ca']
Similar keywords found in Wiki : None
No similar keyword found !!!
Got None after word_match from offline Wiki
Finding 2017 in offline Wiki
near words similar to 2017 : ['2016 ICF Paracanoe World Championships   Women s VL2', '2017 GCC Club Cup']
Similar keywords found in Wiki : ['2017']
Wanted : 2017
Byte start : 327976204, Byte end : 328903379
Page cleaning done... 
Got Title : 2017, 
Cleaned page body : 2017 was designated as International Year of Sustainable Tourism for Development by the United Nations General Assembly.
 
Page url : https://en.wikipedia.org/wiki/2017, 

Fuzzy similarity between 2017 and 2017 is 100


 50%|█████     | 1/2 [00:00<00:00,  1.06it/s]

Page cleaning done... 
Got Title : 2017, 
Cleaned page body : 2017 was designated as International Year of Sustainable Tourism for Development by the United Nations General Assembly.
 
Page url : https://en.wikipedia.org/wiki/2017, 

Fuzzy similarity between 2017 and 2017 is 100
Got {'title': '2017', 'url': 'https://en.wikipedia.org/wiki/2017', 'summary': '2017 was designated as International Year of Sustainable Tourism for Development by the United Nations General Assembly.\n'} after word_match from offline Wiki
Fuzz ration : 100
Got {'title': '2017', 'url': 'https://en.wikipedia.org/wiki/2017', 'summary': '2017 was designated as International Year of Sustainable Tourism for Development by the United Nations General Assembly.\n'} from offline wiki with similarity ration = 100.
checking - 'Rest assured the Democrats do not want you to know that.'
Input has 11 words
Input has 14 tokens
Input has 1 spans
Span boundaries are [[0, 64]]
Candidate entities : ['John F. Kennedy', 'President']


100%|██████████| 2/2 [00:01<00:00,  1.18it/s]

Page cleaning done... 
Got Title : Republican, 
Cleaned page body : Republican can refer to:
 
Page url : https://en.wikipedia.org/wiki/Republican, 

Fuzzy similarity between Republican and Republican is 100
Got {'title': 'Republican', 'url': 'https://en.wikipedia.org/wiki/Republican', 'summary': 'Republican can refer to:\n'} after word_match from offline Wiki
Fuzz ration : 100
Got {'title': 'Republican', 'url': 'https://en.wikipedia.org/wiki/Republican', 'summary': 'Republican can refer to:\n'} from offline wiki with similarity ration = 100.
Entities:
  ('2017', {'url': 'https://en.wikipedia.org/wiki/2017', 'summary': '2017 was designated as International Year of Sustainable Tourism for Development by the United Nations General Assembly.\n'})
  ('John F. Kennedy', {'url': 'https://en.wikipedia.org/wiki/John_F._Kennedy', 'summary': "John Fitzgerald Kennedy (May 29, 1917 – November 22, 1963), often referred to as JFK or Jack, was an American politician who served as the 35th president o

In [ ]:
# kb = KB()
# max_lenn = 1000
# spann = 128

# for text in tqdm(texts):
#     kb = from_text_to_kb(text, "", kb = kb,
#                          verbose=0,
#                          span_length=spann,
#                          max_doc_text=max_lenn,
#                          useWiki=1,
#                          offlineWiki=None)
# kb.print()
    

100%|██████████| 3/3 [01:06<00:00, 22.11s/it]

Entities:
  ('Earth', {'url': 'https://en.wikipedia.org/wiki/Earth', 'summary': "Earth is the third planet from the Sun and the only astronomical object known to harbor life. This is enabled by Earth being a water world, the only one in the Solar System sustaining liquid surface water. Almost all of Earth's water is contained in its global ocean, covering 70.8% of Earth's crust. The remaining 29.2% of Earth's crust is land, most of which is located in the form of continental landmasses within Earth's land hemisphere. Most of Earth's land is somewhat humid and covered by vegetation, while large sheets of ice at Earth's polar deserts retain more water than Earth's groundwater, lakes, rivers and atmospheric water combined. Earth's crust consists of slowly moving tectonic plates, which interact to produce mountain ranges, volcanoes, and earthquakes. Earth has a liquid outer core that generates a magnetosphere capable of deflecting most of the destructive solar winds and cosmic radiation.\n

# LOAD LIAR DATA

In [20]:
liar_path = "./LIAR_data/statements.json"

liar_data = loadFILE(liar_path)

In [28]:
testing_texts = liar_data["fake"][:5]

In [29]:
testing_texts

['The things I said (during the prank call by a blogger posing as GOP contributor David Koch) are the things Ive said publicly all along about the Wisconsin budget debate.',
 'They delayed my swearing-in here in Massachusetts for a couple weeks so they could ram (the health care law) through.',
 '1 percent of candidates that (the National Rifle Association) endorsed in 2012 won.',
 'Were the most highly taxed nation in the world.',
 'I have voted every year in Wisconsin.']

In [37]:
# TAKEN FROM \9th Sem\Fake News Detection\knowledge_graphs\FNN_data\fakenewsnet_dataset\gossipcop\fake\gossipcop-2917215
# LABEL = FAKE

testing_texts = """Fans have always longed for \u201990s golden couple Brad Pitt and Jennifer Aniston to get back together \u2014 something Justin Theroux became keenly aware of two years ago when he found love notes Aniston had saved from the heartthrob, a source recalls in the new issue of Us Weekly.\n\nRelated: Jennifer Aniston and Justin Theroux: The Way They Were Oh, how sweet it is! Take a look back at Jennifer Aniston and Justin Theroux's romance -- from their engagement, sexy getaways, annual star-studded holiday parties and more!\n\n\u201cHe stumbled upon old Post-it notes Brad had written,\u201d a source tells Us. \u201cSweet little Post-its like, \u2018You looked nice tonight\u2019 or \u2018Miss you already.\u2019\u201d\n\nThe source explains that impact of those little notes was huge. \u201cJen assured him they weren\u2019t a big deal, but Justin wasn\u2019t thrilled \u2026 Justin had moments of insecurity like that.\u201d\n\nRelated: Jennifer Aniston and Justin Theroux Split: Revisit Their Sweetest Quotes About Each Other They once had it all! Jennifer Aniston and Justin Theroux have called it quits on their marriage after more than two years of marriage. The A-list couple confirmed their split in a statement to Us Weekly on Thursday, February 15. \u201cIn an effort to reduce any further speculation, we have decided to announce our separation. [\u2026]\n\nAs previously reported, Aniston, 49, and Theroux, 46, announced their separation on Thursday, February 15, via a joint statement: \u201cIn an effort to reduce any further speculation, we have decided to announce our separation. This decision was mutual and lovingly made at the end of last year. We are two best friends who have decided to part ways as a couple, but look forward to continuing our cherished friendship.\u201d\n\nThe duo, who started dating in May 2011, tied the knot in August 2015. While it was Theroux\u2019s first marriage, Aniston was married to Pitt, 54, from 2000 to 2005. He infamously moved on from the Friends alum with his Mr. and Mrs. Smith costar, Angelina Jolie. Pitt and Jolie, who share Maddox, 16, Pax, 14, Zahara, 12, Shiloh, 11 and 9-year-old twins Knox and Vivienne, called it quits in September 2016 after two years of marriage and 12 years together.\n\n\u201cJen has struggled with the perception that she is this pathetic woman after the divorce from Brad,\u201d the insider tells Us. \u201cIt played a role in her wanting to marry Justin.\u201d\n\nFor more on Aniston and Theroux\u2019s split, pick up the new issue of Us Weekly, on stands now!"""

# testing_texts = testing_texts.split(". ")
testing_texts = sent_tokenize(testing_texts)


In [81]:
# TAKEN FROM https://www.boomlive.in/fact-check/viral-quote-ladakh-mp-jamyang-tsering-namgyal-apology-joining-bjp-supporting-modi-claim-social-media-25050
# LABEL = FALSE 

testing_texts = """Breaking - Apology from a BJP sitting MP. 
"It was my worst decision to join the BJP and support Narendra Modi. 
 I was not aware of his tactics, people of Ladakh please forgive me. "

- Jamyang Tsering (BJP sitting MP from Ladakh)"""

# testing_texts = testing_texts.split(". ")
testing_texts = sent_tokenize(testing_texts)


In [47]:
# GENERATED FROM CHATGPT
# LABEL = FALSE

testing_texts = """In a grounbreaking revelation, Naruto's creator Masashi Kishimoto discloses plans for an alternate ending where Naruto joins forces with the Akatuski. This shocking twist, thought never realized, sparks speculation about the series' untapped potential and complex nature of heroism."""

testing_texts = testing_texts.split(". ")

In [54]:
# GENERATED FROM CHATGPT
# 

testing_texts = """ In the world of ninjas, dreams are as crucial as kunai. Naruto Uzumaki's aspiration burns bright—to become the Hokage, a title etched in history. His rival, Sasuke Uchiha, walks a parallel path, yearning to eclipse all shadows. Sakura Haruno, with her blossoming skills in medical ninjutsu, stands as a pillar of strength. But tales sometimes twist; Kakashi Hatake's Sharingan, not a relic from Obito, conceals mysteries. Legends echo in the village—Minato Namikaze, the Fourth Hokage, master of time and space. And in the shadows lurks Jiraiya, mentor to Naruto, guardian of sage wisdom."""

# testing_texts = testing_texts.split(". ")
testing_texts = sent_tokenize(testing_texts)

In [38]:
testing_texts

['Fans have always longed for ’90s golden couple Brad Pitt and Jennifer Aniston to get back together — something Justin Theroux became keenly aware of two years ago when he found love notes Aniston had saved from the heartthrob, a source recalls in the new issue of Us Weekly.',
 'Related: Jennifer Aniston and Justin Theroux: The Way They Were Oh, how sweet it is!',
 "Take a look back at Jennifer Aniston and Justin Theroux's romance -- from their engagement, sexy getaways, annual star-studded holiday parties and more!",
 '“He stumbled upon old Post-it notes Brad had written,” a source tells Us.',
 '“Sweet little Post-its like, ‘You looked nice tonight’ or ‘Miss you already.’”\n\nThe source explains that impact of those little notes was huge.',
 '“Jen assured him they weren’t a big deal, but Justin wasn’t thrilled … Justin had moments of insecurity like that.”\n\nRelated: Jennifer Aniston and Justin Theroux Split: Revisit Their Sweetest Quotes About Each Other They once had it all!',
 'J

In [39]:
kb = KB()
max_lenn = 1000
spann = 64

for text in tqdm(testing_texts):
    kb = from_text_to_kb(text, "", kb = kb,
                         verbose=0,
                         span_length=spann,
                         max_doc_text=max_lenn,
                         useWiki=1,
                         offlineWiki=offline_wikipedia)
kb.print()

 76%|███████▋  | 13/17 [00:49<00:09,  2.28s/it]e:\Downloads\WeLearn\9th Sem\nineSemEnv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file e:\Downloads\WeLearn\9th Sem\nineSemEnv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 17/17 [01:11<00:00,  4.20s/it]

Entities:
  ('Brad Pitt', {'url': 'https://en.wikipedia.org/wiki/Brad_Pitt', 'summary': "William Bradley Pitt (born December 18, 1963) is an American actor and film producer. He is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, two Golden Globe Awards, and a Primetime Emmy Award. As a public figure, Pitt has been cited as one of the most powerful and influential people in the American entertainment industry. Pitt first gained recognition as a cowboy hitchhiker in the Ridley Scott road film Thelma & Louise (1991). His first leading roles in big-budget productions came with the drama films A River Runs Through It (1992) and Legends of the Fall (1994). He also starred in the horror film Interview with the Vampire (1994), alongside Tom Cruise. He gave critically acclaimed performances in David Fincher's crime thriller Seven (1995) and the science fiction film 12 Monkeys (1995). The latter earned him a Golden Globe Award for Best Supportin

# RAG

In [48]:
entities_in_kb = kb.entities
doc_titles = []
docs = {}
for _entt, _values in entities_in_kb.items():
    _url = _values['url']
    _url_word = _url.split("wiki/")[-1].strip().replace("_", " ")
    print(_url_word)
    # doc_titles.append(_url_word) 
    # print(offline_wikipedia.word_match(_url_word, verbose=0, summaryOnly=False))
    docs[_url_word] = offline_wikipedia.word_match(_url_word, verbose=0, summaryOnly=False)
    if docs[_url_word]:
        docs[_url_word] = docs[_url_word]["summary"]
    else: 
        del docs[_url_word] # REMOVING THOSE ENTRIES WHICH CONTAIN None
        print(f"---{_url_word} removed")
    # --- NEED TO FIX THIS ---
    # try:
    #     docs[_url_word] = wikipedia.page(_url_word, auto_suggest=False).content.replace("\n\n", " ") # <--- NEED TO FIX THIS
    # except wikipedia.exceptions.DisambiguationError :
    #     pass

Brad Pitt
Jennifer Aniston
Justin Theroux
American Woman (album)
---American Woman (album) removed
Engagement
Brad
Post-it note
Miss You Already
A-list
Us Weekly
2008 Summer Olympics
2008
2012 Summer Olympics
2012
2008 FIFA U-20 Women's World Cup squads
Aniston
Theroux
Last Year
Last Christmas
2009
2014
2016 Summer Olympics
2016
57th Annual Grammy Awards
2015
Pitt
2000
Friends
Angelina Jolie
Maddox
Pax
Zahara
Shiloh
Knox
Vivian (personal name)
Jen
The Divorce
Justin


In [49]:
docs

{'Brad Pitt': 'William Bradley Pitt (born December 18, 1963) is an American actor and film producer. He is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, two Golden Globe Awards, and a Primetime Emmy Award. As a public figure, Pitt has been cited as one of the most powerful and influential people in the American entertainment industry. Pitt first gained recognition as a cowboy hitchhiker in the Ridley Scott road film Thelma & Louise (1991). His first leading roles in big-budget productions came with the drama films A River Runs Through It (1992) and Legends of the Fall (1994). He also starred in the horror film Interview with the Vampire (1994), alongside Tom Cruise. He gave critically acclaimed performances in David Fincher\'s crime thriller Seven (1995) and the science fiction film 12 Monkeys (1995). The latter earned him a Golden Globe Award for Best Supporting Actor and his first Academy Award nomination.\nPitt found greater comme

In [50]:
data = ""
data = "".join([f"{_content}\n\n" for _word, _content in docs.items()])
print(data)


William Bradley Pitt (born December 18, 1963) is an American actor and film producer. He is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, two Golden Globe Awards, and a Primetime Emmy Award. As a public figure, Pitt has been cited as one of the most powerful and influential people in the American entertainment industry. Pitt first gained recognition as a cowboy hitchhiker in the Ridley Scott road film Thelma & Louise (1991). His first leading roles in big-budget productions came with the drama films A River Runs Through It (1992) and Legends of the Fall (1994). He also starred in the horror film Interview with the Vampire (1994), alongside Tom Cruise. He gave critically acclaimed performances in David Fincher's crime thriller Seven (1995) and the science fiction film 12 Monkeys (1995). The latter earned him a Golden Globe Award for Best Supporting Actor and his first Academy Award nomination.
Pitt found greater commercial success sta

### RETRIVER

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

SPLIT LARGE TEXT

In [51]:
# SPLITTING LARGE DOCS IN SMALLER CHUNKS
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_text(data)
chunks


['William Bradley Pitt (born December 18, 1963) is an American actor and film producer. He is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, two Golden Globe Awards, and a Primetime Emmy Award. As a public figure, Pitt has been cited as one of the',
 'public figure, Pitt has been cited as one of the most powerful and influential people in the American entertainment industry. Pitt first gained recognition as a cowboy hitchhiker in the Ridley Scott road film Thelma & Louise (1991). His first leading roles in big-budget productions came with the',
 "roles in big-budget productions came with the drama films A River Runs Through It (1992) and Legends of the Fall (1994). He also starred in the horror film Interview with the Vampire (1994), alongside Tom Cruise. He gave critically acclaimed performances in David Fincher's crime thriller Seven",
 "in David Fincher's crime thriller Seven (1995) and the science fiction film 12 Monkeys (1995). T

VECTORIZE AND STORE 

In [52]:
# MAKING TEXT EMBEDDING (VECTORIZING CHUNKS)
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': device}

encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
                                    model_name=modelPath, 
                                    model_kwargs=model_kwargs, 
                                    encode_kwargs=encode_kwargs 
                                    )


In [53]:

vector_store = FAISS.from_texts(chunks, embedding=embeddings)

In [27]:
# statement.split('. ')

In [28]:
# user_qns = statement.split(". ")

In [50]:
# user_qn = ""
# relevant_doc = []
# for i in user_qns[:3]:
#     user_qn = i 
#     relevant_doc = vector_store.similarity_search(user_qn, k = 3)

In [51]:
# relevant_doc

[Document(page_content='The tiger (Panthera tigris) is the largest living cat species and a member of the genus Panthera native to Asia. It has a powerful, muscular body with a large head and paws, a long tail, and distinctive black, mostly vertical stripes on orange fur. It was first scientifically described in 1758 and'),
 Document(page_content='=== Hunting and diet === The tiger is a carnivore and an apex predator feeding mainly on ungulates, with a particular preference for sambar deer, Manchurian wapiti, barasingha and wild boar. Tigers kill large ungulates like gaur and opportunistically, smaller prey like monkeys, peafowl and other'),
 Document(page_content='sequencing of 32 samples support six monophyletic tiger clades corresponding with the six living proposed subspecies and indicate they descended from a common ancestor around 110,000 years ago. Studies in 2021 and 2023 also affirmed the genetic distinctiveness and separation of these tigers.')]

### PREPARE MODEL

In [29]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM
from transformers import pipeline, set_seed
from transformers import T5Tokenizer, T5ForConditionalGeneration

from llama3 import llama3
from nltk import sent_tokenize

### llama3

In [30]:
llama3_rag_model = llama3()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
# user_qn

In [35]:
# relevant_doc

In [36]:
# contextt = '\n'.join(['[DOC ' + str(i) + '] : '+docc.page_content for i, docc in enumerate(relevant_doc)])

In [37]:
# prompt = f"""<<CONTEXT>>\n{contextt}\n\n<<CHECK>> {user_qn}"""

In [38]:
# print(prompt)

In [39]:
# llama3_rag_model.llama3_summary(user_qn, 
#                                 contextt, 
#                                 verbose=1)

In [40]:
# user_qns

------ LOOP -------

In [58]:
def clean_output(llama_response):
    possible_response = ["true", "false", "pantsonfire"]
    llama_response = llama_response.lower()
    llama_responses = [s.strip() for s in llama_response.split()]
    if len(llama_responses) > 4 \
                        and \
                        sum([llama_responses.count(resp) for resp in possible_response]) <= 1:
        # MAKE SURE THAT THE RESPONSE DOES NOT CONTAIN GARBAGE
        # AND "TRUE", "FALSE" AND "PANTSONFIRE" ANY OF THEM APPEARS ONLY ONCE 
        return None
    if "true" in llama_responses:
        return 1
    elif "false" in llama_responses:
        return 0
    elif "pantsonfire" in llama_responses:
        return -1
    else:
        return None
    
def calc_truth(truth_dict):
    total_count = len(truth_dict)
    
    frac_1 = sum(1 for value in truth_dict.values() if value == 1)/total_count if total_count > 0 else 0
    frac_0 = sum(1 for value in truth_dict.values() if value == 0)/total_count if total_count > 0 else 0
    frac_minus_1 = sum(1 for value in truth_dict.values() if value == -1)/total_count if total_count > 0 else 0

    return {
        "True" : frac_1,
        "False" : frac_0,
        "PantsOnFire" : frac_minus_1
    }




In [54]:
testing_texts

['Fans have always longed for ’90s golden couple Brad Pitt and Jennifer Aniston to get back together — something Justin Theroux became keenly aware of two years ago when he found love notes Aniston had saved from the heartthrob, a source recalls in the new issue of Us Weekly.',
 'Related: Jennifer Aniston and Justin Theroux: The Way They Were Oh, how sweet it is!',
 "Take a look back at Jennifer Aniston and Justin Theroux's romance -- from their engagement, sexy getaways, annual star-studded holiday parties and more!",
 '“He stumbled upon old Post-it notes Brad had written,” a source tells Us.',
 '“Sweet little Post-its like, ‘You looked nice tonight’ or ‘Miss you already.’”\n\nThe source explains that impact of those little notes was huge.',
 '“Jen assured him they weren’t a big deal, but Justin wasn’t thrilled … Justin had moments of insecurity like that.”\n\nRelated: Jennifer Aniston and Justin Theroux Split: Revisit Their Sweetest Quotes About Each Other They once had it all!',
 'J

In [33]:
text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=20)
_testing_texts = text_splitter2.split_text(". ".join(testing_texts))

In [55]:
sentence_truth = {}
for sent_no, user_qn in tqdm(enumerate(testing_texts), desc="Sentences"):

    print("--- Fetching relevant docs...")
    
    relevant_docs = vector_store.similarity_search(user_qn, k = 5)
    print("--- Relevant docs fetched...")

    contextt = '\n'.join(['[DOC ' + str(i) + '] : '+ docc.page_content 
                                            for i, docc in enumerate(relevant_docs)])
    
    prompt = f"""<<CONTEXT>>\n{contextt}\n\n<<CHECK>> {user_qn}"""
    print(prompt)
    
    
    llama_response = llama3_rag_model.llama3_summary(user_qn, 
                                                    contextt, 
                                                    verbose=1)
    
    print("response :", llama_response)
    
    sentence_truth[sent_no] = clean_output(llama_response)
    print(sentence_truth)


Sentences: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--- Fetching relevant docs...
--- Relevant docs fetched...
<<CONTEXT>>
[DOC 0] : Aniston met Brad Pitt in 1998, and their relationship was highly publicized. She married Pitt, after two years of dating, on July 29, 2000, in a lavish Malibu wedding. For a few years, their marriage was considered the rare Hollywood success. On January 7, 2005, they announced their separation, and
[DOC 1] : Pitt met actress Jennifer Aniston in 1998; they married in a private wedding ceremony in Malibu on July 29, 2000. In January 2005, Pitt and Aniston announced they had decided to separate. Two months later, Aniston filed for divorce, citing irreconcilable differences. Pitt and Aniston's divorce was
[DOC 2] : 7, 2005, they announced their separation, and finalized their divorce on October 2. During their divorce proceedings, news media speculated that Pitt had been unfaithful to Aniston with his Mr. & Mrs. Smith co-star Angelina Jolie, with whom he began a relationship soon after the split. In the
[DOC 3

Sentences: 1it [00:15, 15.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 --- Got some response --- 
 

 --- Returning after decoding response... --- 

response : True
{0: 1}
--- Fetching relevant docs...
--- Relevant docs fetched...
<<CONTEXT>>
[DOC 0] : Aniston began a relationship with actor, director, and screenwriter Justin Theroux in May 2011. The following January, they purchased a home in the Bel Air neighborhood of Los Angeles for roughly $22&nbsp;million. They became engaged on August 10, 2012 and were married on August 5, 2015, at their
[DOC 1] : married on August 5, 2015. On February 15, 2018, Theroux and Aniston announced they had separated at the end of 2017.
[DOC 2] : Theroux was in a relationship with stylist and costume designer Heidi Bivens from 1997 until 2011. Theroux began dating actress Jennifer Aniston in 2011 after working with her on the film Wanderlust. They became engaged in August 2012, and got married on August 5, 2015. On February 15, 2018,
[DOC 3] : Film Festival called her performance "heartbreakingly good", Gregory Ellwood 

Sentences: 2it [00:45, 24.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


 --- Got some response --- 
 

 --- Returning after decoding response... --- 

response : True
{0: 1, 1: -1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: -1, 8: None, 9: -1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1}
--- Fetching relevant docs...
--- Relevant docs fetched...
<<CONTEXT>>
[DOC 0] : In 2005, amid reports that their divorce was due to Aniston's refusal to have children with Pitt, Aniston said, "I've never in my life said I didn't want to have children. I did and I do and I will! ... I would never give up that experience for a career." Aniston said the divorce prompted her to
[DOC 1] : (1999). Aniston said she had been devastated by the death of her longtime therapist, whom she credited for helping make her separation from Pitt easier. She said she did not regret her relationship with Pitt, describing it as "very intense" and "a beautiful, complicated relationship".
[DOC 2] : 7, 2005, they announced their separation, and finalized their divorce on October 2. During their divorce proceedings, 

Sentences: 16it [07:18, 19.58s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 --- Got some response --- 
 

 --- Returning after decoding response... --- 

response : True
{0: 1, 1: -1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: -1, 8: None, 9: -1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}
--- Fetching relevant docs...
--- Relevant docs fetched...
<<CONTEXT>>
[DOC 0] : Aniston began a relationship with actor, director, and screenwriter Justin Theroux in May 2011. The following January, they purchased a home in the Bel Air neighborhood of Los Angeles for roughly $22&nbsp;million. They became engaged on August 10, 2012 and were married on August 5, 2015, at their
[DOC 1] : married on August 5, 2015. On February 15, 2018, Theroux and Aniston announced they had separated at the end of 2017.
[DOC 2] : began a relationship soon after the split. In the months following, the public's reaction towards the divorce was reported in the press, and "Team Aniston" and "Team Jolie" T-shirts appeared throughout the country. Aniston commented on the divorce in a January 2015 interview wi

Sentences: 17it [07:54, 27.89s/it]


 --- Got some response --- 
 

 --- Returning after decoding response... --- 

response : PantsOnFire
{0: 1, 1: -1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: -1, 8: None, 9: -1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: -1}


In [56]:
sentence_truth

{0: 1,
 1: -1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: -1,
 8: None,
 9: -1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: -1}

In [59]:
calc_truth(sentence_truth)

{'True': 0.7058823529411765, 'False': 0.0, 'PantsOnFire': 0.23529411764705882}